In [1]:
%load_ext autoreload
%autoreload 2

# Making systems

In [2]:
import molsysmt as msm

In [3]:
from molsysmt import systems
import numpy as np

In [4]:
molsys = systems['T4 lysozyme L99A']['t4_lysozyme_L99A.h5msm']

In [5]:
molsys = msm.convert(molsys, to_form='molsysmt.MolSys')

In [6]:
msm.get(molsys, n_bonds=True)

2645

In [7]:
msm.build.remove_bonds(molsys)

In [8]:
%%time
msm.build.add_missing_bonds(molsys)

CPU times: user 2.34 s, sys: 47.4 ms, total: 2.38 s
Wall time: 2.38 s


In [9]:
msm.get(molsys, n_bonds=True)

2645

In [ ]:
len(bb)

In [ ]:
msm.info(molsys_ref, element='atom', selection=[0,1,2,3,4,5,6,7,8,9])

In [ ]:
for ii in aa:
    if ii not in bb:
        print(ii)

In [ ]:
molsys = msm.convert(molsys, to_form='nglview.NGLWidget')

In [ ]:
n_molecules_ref, n_structures_ref = msm.get(molsys_ref, element='system', n_molecules=True, n_structures=True)

In [ ]:
msm.convert(molsys, to_form='molsysmt.Topology', get_missing_bonds=True)

In [ ]:
msm.get(molsys, element='system', n_molecules=True)

In [ ]:
n_molecules, n_structures = msm.get(molsys, element='system', n_molecules=True, n_structures=True)

In [ ]:
if []:
    print('si')